In [6]:
# %pip install python-dotenv

In [29]:
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

# Access an environment variable
endpoint = os.getenv("endpoint")
key = os.getenv("key")

# print(f"Endpoint: {endpoint}")
# print(f"Key: {key}")
print("Environment variables loaded successfully.")

Environment variables loaded successfully.


In [3]:
import requests

analyzerId = "image_analyzer_01"  # Replace with your actual analyzer ID
url = f"{endpoint}/contentunderstanding/analyzers/{analyzerId}?api-version=2024-12-01-preview"
headers = {
    "Ocp-Apim-Subscription-Key": key,
    "Content-Type": "application/json"
}

with open('image_request_body.json', 'r') as file:
    data = file.read()

response = requests.put(url, headers=headers, data=data)

print(response.status_code)
print(response.json())

201
{'analyzerId': 'image_analyzer_01', 'description': 'Sample chart analyzer', 'createdAt': '2025-02-22T03:32:08Z', 'lastModifiedAt': '2025-02-22T03:32:08Z', 'config': {'returnDetails': False, 'disableContentFiltering': False}, 'fieldSchema': {'fields': {'Title': {'type': 'string'}, 'ChartType': {'type': 'string', 'method': 'classify', 'enum': ['bar', 'line', 'pie']}}}, 'warnings': [], 'status': 'ready', 'scenario': 'image'}


In [30]:
# print(response.headers)
operation_location = response.headers.get('Operation-Location')
# print(f"Operation-Location: {operation_location}")

In [5]:
operationId = "your_operation_id"  # Replace with your actual operation ID
url = operation_location

response = requests.get(url, headers=headers)

print(response.status_code)
print(response.json())

200
{'id': '91dd7041-7127-46a8-b6fb-7ea2e693d6b2', 'status': 'Succeeded', 'result': {'analyzerId': 'image_analyzer_01', 'description': 'Sample chart analyzer', 'createdAt': '2025-02-22T03:32:08Z', 'lastModifiedAt': '2025-02-22T03:32:08Z', 'config': {'returnDetails': False, 'disableContentFiltering': False}, 'fieldSchema': {'fields': {'Title': {'type': 'string'}, 'ChartType': {'type': 'string', 'method': 'classify', 'enum': ['bar', 'line', 'pie']}}}, 'warnings': [], 'status': 'ready', 'scenario': 'image'}}


In [7]:
from IPython.display import Image, display

file_url = "https://images.plot.ly/excel/grouped-bar-chart/final-chart.jpg"

display(Image(url=file_url))


In [13]:

analyze_url = f"{endpoint}/contentunderstanding/analyzers/{analyzerId}:analyze?api-version=2024-12-01-preview"
payload = {"url": file_url}

response = requests.post(analyze_url, headers=headers, json=payload)

print(response.status_code)
print(response.json())
print(response.json().get("status"))


202
{'id': '935bfe33-9036-4675-8e10-ced9d3a9c62f', 'status': 'Running', 'result': {'analyzerId': 'image_analyzer_01', 'apiVersion': '2024-12-01-preview', 'createdAt': '2025-02-22T03:35:41Z', 'warnings': [], 'contents': []}}
Running


In [31]:
operation_location = response.headers.get('Operation-Location')
# print(f"Operation-Location: {operation_location}")

In [14]:
response = requests.get(operation_location, headers=headers)

print(response.status_code)
print(response.json().get("status"))

200
Succeeded


In [15]:
import json

# resultId = "c62115c9-090e-4fa2-b326-e0eac6f0e0be"  # Replace with your actual result ID
results_url = operation_location

response = requests.get(results_url, headers=headers)

print(response.status_code)

json_response = response.json()
print(json.dumps(json_response, indent=4))

200
{
    "id": "91dd7041-7127-46a8-b6fb-7ea2e693d6b2",
    "status": "Succeeded",
    "result": {
        "analyzerId": "image_analyzer_01",
        "description": "Sample chart analyzer",
        "createdAt": "2025-02-22T03:32:08Z",
        "lastModifiedAt": "2025-02-22T03:32:08Z",
        "config": {
            "returnDetails": false,
            "disableContentFiltering": false
        },
        "fieldSchema": {
            "fields": {
                "Title": {
                    "type": "string"
                },
                "ChartType": {
                    "type": "string",
                    "method": "classify",
                    "enum": [
                        "bar",
                        "line",
                        "pie"
                    ]
                }
            }
        },
        "warnings": [],
        "status": "ready",
        "scenario": "image"
    }
}


In [32]:
# analyze file 
analyze_url = f"{endpoint}/contentunderstanding/analyzers/{analyzerId}:analyze?api-version=2024-12-01-preview"
response = requests.post(analyze_url, headers=headers, json={"url": file_url})

print(response.status_code)
# print(response.json())

analyze_job_url = response.headers.get('Operation-Location')
# print(f"Analyze Job URL: {analyze_job_url}")

202


In [25]:
# get analyzer job results

response = requests.get(analyze_job_url, headers=headers)
print(response.status_code)
json_response = response.json()
print(json.dumps(json_response, indent=4))

200
{
    "id": "65ac5367-73ab-4d02-a753-232108199bd5",
    "status": "Succeeded",
    "result": {
        "analyzerId": "image_analyzer_01",
        "apiVersion": "2024-12-01-preview",
        "createdAt": "2025-02-22T03:41:12Z",
        "warnings": [],
        "contents": [
            {
                "markdown": "![image](image)\n",
                "fields": {
                    "Title": {
                        "type": "string",
                        "valueString": "Grouped Bar Chart"
                    },
                    "ChartType": {
                        "type": "string",
                        "valueString": "bar"
                    }
                },
                "kind": "document",
                "startPageNumber": 1,
                "endPageNumber": 1,
                "unit": "pixel",
                "pages": [
                    {
                        "pageNumber": 1
                    }
                ]
            }
        ]
    }
}


In [26]:
from pydantic import BaseModel
from typing import List, Optional, Dict, Any

class Item(BaseModel):
    Description: Optional[str]
    Amount: Optional[float]

class FieldSchema(BaseModel):
    VendorName: Optional[str]
    Items: Optional[List[Item]]

class Result(BaseModel):
    analyzerId: str
    apiVersion: str
    createdAt: str
    warnings: Optional[List[Any]]
    contents: Optional[List[Any]]

class JsonResponse(BaseModel):
    id: str
    status: str
    result: Result

# Convert json_response to Pydantic object
pydantic_response = JsonResponse(**json_response)
print(pydantic_response)

id='65ac5367-73ab-4d02-a753-232108199bd5' status='Succeeded' result=Result(analyzerId='image_analyzer_01', apiVersion='2024-12-01-preview', createdAt='2025-02-22T03:41:12Z', warnings=[], contents=[{'markdown': '![image](image)\n', 'fields': {'Title': {'type': 'string', 'valueString': 'Grouped Bar Chart'}, 'ChartType': {'type': 'string', 'valueString': 'bar'}}, 'kind': 'document', 'startPageNumber': 1, 'endPageNumber': 1, 'unit': 'pixel', 'pages': [{'pageNumber': 1}]}])


In [27]:
pydantic_response.result.contents[0].keys()


dict_keys(['markdown', 'fields', 'kind', 'startPageNumber', 'endPageNumber', 'unit', 'pages'])

In [28]:
pydantic_response.result.contents[0]['fields']

{'Title': {'type': 'string', 'valueString': 'Grouped Bar Chart'},
 'ChartType': {'type': 'string', 'valueString': 'bar'}}